In [1]:
import gameEngine
import copy
import pickle
import concurrent.futures
from random import choice
from random import randint
from tqdm import tqdm

In [2]:
def run_single_game(p1_func,p2_func):
    s = gameEngine.gen_starting_state(2)
    latch = True
    p1_win = 0
    p2_win = 0
    while latch:
        action_list = gameEngine.gen_valid_moves(s)
        if len(action_list) == 0:
            s[1] = 'lose'
        else:
            if s[0] == 'p1':
                if p1_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p1_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            else:
                if p2_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p2_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            gameEngine.next_state(s,action_list,selected_index)
        if s[1] == 'win':
            if s[0] == 'p1':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
        elif s[1] == 'lose':
            if s[0] == 'p2':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
    return (p1_func,p2_func,p1_win,p2_win)

In [2]:
def run_single_game_save(p1_func,p2_func,game_id):
    s = gameEngine.gen_starting_state(2)
    s_hist = [copy.deepcopy(s)]
    latch = True
    p1_win = 0
    p2_win = 0
    selected_index = -1
    while latch:
        action_list = gameEngine.gen_valid_moves(s)
        heuristic_list = []
        if len(action_list) == 0:
            s[1] = 'lose'
        else:
            if s[0] == 'p1':
                if p1_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p1_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            else:
                if p2_func == 0:
                    selected_index = randint(0, len(action_list)-1)
                elif p2_func == 1:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
                else:
                    heuristic_list = []
                    for i in range(len(action_list)):
                        heuristic_list.append(gameEngine.f_heuristic_god(s,action_list,i))
                    max_value = max(heuristic_list)
                    max_value_indexes = [index for index, value in enumerate(heuristic_list) if value == max_value]
                    selected_index = choice(max_value_indexes)
            s_temp = copy.deepcopy(s)
            s_temp.append(selected_index)
            s_temp.append(action_list)
            s_temp.append(heuristic_list)
            s_hist.append(s_temp)
            gameEngine.next_state(s,action_list,selected_index)
        if s[1] == 'win':
            if s[0] == 'p1':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
        elif s[1] == 'lose':
            if s[0] == 'p2':
                p1_win = p1_win + 1
            else:
                p2_win = p2_win + 1
            latch = False
        if latch == False:
            s_temp = copy.deepcopy(s)
            s_hist.append(s_temp)
    filename = f"saved_state/game_state_{p1_func}_{p2_func}_{game_id}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(s_hist, file)
    return (p1_func,p2_func,p1_win,p2_win)

In [4]:
function_name = ['random','norm heuristic','god heuristic']
function_arr = [0,1,2]
game_max = 100
player_pairs = []
for p1 in range(len(function_arr)):
    for p2 in range(len(function_arr)):
        for i in range(game_max):
            player_pairs.append((p1,p2))

In [5]:
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    results = list(tqdm(executor.map(run_single_game, *zip(*player_pairs)), total=len(player_pairs)))

100%|██████████| 900/900 [00:00<00:00, 2838.40it/s]


In [3]:
function_name = ['random','norm heuristic','god heuristic']
function_arr = [0,1,2]
game_max = 100
player_pairs = []
for p1 in range(len(function_arr)):
    for p2 in range(len(function_arr)):
        for i in range(game_max):
            player_pairs.append((p1,p2, i))

In [4]:
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    results = list(tqdm(executor.map(run_single_game_save, *zip(*player_pairs)), total=len(player_pairs)))

100%|██████████| 900/900 [00:00<00:00, 1202.16it/s]


In [5]:
results

[(0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0, 1),
 (0, 0, 0,

In [10]:
sum_arr = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
sum_cnt_p1 = [0,0,0,0,0,0,0,0,0]
sum_cnt_p2 = [0,0,0,0,0,0,0,0,0]
for i in range(len(sum_arr)):
    for j in range(len(results)):
        if sum_arr[i][0]==results[j][0] and sum_arr[i][1]==results[j][1]:
            if results[j][2]==1:
                sum_cnt_p1[i] = sum_cnt_p1[i] + 1
            else:
                sum_cnt_p2[i] = sum_cnt_p2[i] + 1
function_name = ['random','norm heuristic','god heuristic']
for i in range(len(sum_arr)):
    print('-----------')
    print('p1 func:'+function_name[sum_arr[i][0]])
    print('p2 func:'+function_name[sum_arr[i][1]])
    print('p1 win :'+str(sum_cnt_p1[i]))
    print('p2 win :'+str(sum_cnt_p2[i]))

-----------
p1 func:random
p2 func:random
p1 win :51
p2 win :49
-----------
p1 func:random
p2 func:norm heuristic
p1 win :0
p2 win :100
-----------
p1 func:random
p2 func:god heuristic
p1 win :1
p2 win :99
-----------
p1 func:norm heuristic
p2 func:random
p1 win :100
p2 win :0
-----------
p1 func:norm heuristic
p2 func:norm heuristic
p1 win :49
p2 win :51
-----------
p1 func:norm heuristic
p2 func:god heuristic
p1 win :61
p2 win :39
-----------
p1 func:god heuristic
p2 func:random
p1 win :99
p2 win :1
-----------
p1 func:god heuristic
p2 func:norm heuristic
p1 win :42
p2 win :58
-----------
p1 func:god heuristic
p2 func:god heuristic
p1 win :55
p2 win :45
